In [1]:
!pip install pyarrow zstandard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 38.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
import pyarrow.feather as feather
import zstandard

import pyarrow.feather as feather
# CONTENT BASED EMBEDDINGS
# Specify the path to your .feather.zstd file
file_path = '/content/drive/Shareddrives/CSCE 470 Project/recomm.feather.zstd'

# Read the Feather file using pyarrow (which automatically handles decompression)
table = feather.read_table(file_path)

# Convert the table to a pandas DataFrame for easier viewing
all_recomm_embeds = table.to_pandas()

# Print or manipulate the DataFrame as needed
print(all_recomm_embeds)
# James Charles UCucot-Zp428OwkyRm2I7v2Q
c = all_recomm_embeds[all_recomm_embeds['channelId'] == 'UCucot-Zp428OwkyRm2I7v2Q']
# there is no embedding for james charles

                      channelId         0         1         2         3  \
0      UC--3c8RqSfAqYBdDjIG3UNA  0.030411 -0.217000  0.144818  0.342317   
1      UC--3ozenTzry39-xMqAPQlw -0.304254 -0.579989 -0.366970  0.484797   
2      UC--3pN-wtQrC48GAK8wsBsg -0.234844 -0.043223 -0.384786  0.524040   
3      UC--70ql_IxJmhmqXqrkJrWQ  0.098866  0.052671  0.046164  0.291882   
4      UC--EwQJeJ6SKlJdswehQH4g -0.377770 -0.177363  0.508269  0.562063   
...                         ...       ...       ...       ...       ...   
43769  UCzzamhkOmpSnZ7BSBikvKnA  0.181688 -0.326179  0.065002  0.511614   
43770  UCzzfu83LhPMMuhtDVA75rfw  0.230297 -0.242405  0.188479  0.800248   
43771  UCzzsltdnAgTMSlPOvjko_Dw  0.340042  0.415698  0.389988  0.602569   
43772  UCzztaOtjmaZoiLvWLagq7bg  0.130557 -0.204882  0.180883  0.378668   
43773  UCzzzUN8yvD2LRAnY-lhzyLQ  0.112501  0.003378 -0.156830  0.262225   

              4         5         6         7         8  ...        54  \
0     -0.111861  0.061309

In [7]:
#extract embeds of only our top 1000

chan_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chanIDs.csv'

chan_ids = pd.read_csv(chan_file_path)

merged_df = pd.merge(all_recomm_embeds, chan_ids, left_on='channelId', right_on='Youtube Url', how='inner')

# Drop the redundant column
merged_df = merged_df.drop(columns='Youtube Url')

matching_chan_df = merged_df['channelId']

In [8]:
from sklearn.metrics.pairwise import cosine_similarity


In [9]:

# Function to get recommendations based on cosine similarity
def get_channel_recommendations(channel_id, embeddings_df, top_n=5):
    # Find the index of the given channel
    channel_index = embeddings_df.index[embeddings_df['channelId'] == channel_id].tolist()[0]

    # Select the embeddings for the given channel
    channel_embeddings = embeddings_df.iloc[channel_index, 1:]  # Assuming embeddings start from the second column

    # Reshape the embeddings to a 2D array
    channel_embeddings = channel_embeddings.values.reshape(1, -1)

    # Calculate cosine similarity between the given channel and all other channels
    similarity_scores = cosine_similarity(channel_embeddings, embeddings_df.iloc[:, 1:])

    # Get indices of top_n most similar channels (excluding the channel itself)
    similar_channel_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]

    # Get the channel_id of recommended channels
    recommended_channels = embeddings_df['channelId'].iloc[similar_channel_indices].tolist()

    return recommended_channels



In [10]:

# good test case is ColdplayVEVO UCDPM_n1atn2ijUwHd0NNRQw
chan_recommendations_dict = {}
for chan in matching_chan_df:
  recommendations = get_channel_recommendations(chan, merged_df)
  chan_recommendations_dict[chan] = recommendations
chan_recommendations_dict

{'UC-0D1RlSzcvBoODCKSoLl2g': ['UCgFXm4TI8htWmCyJ6cVPG_A',
  'UCVbPnm-qCjH-8Yl1-k4BQkA',
  'UCk8GzjMOrta8yxDcKfylJYw',
  'UChGJGhZ9SOOHvBB0Y4DOO_w',
  'UCx790OVgpTC1UVBQIqu3gnQ'],
 'UC-2Y8dQb0S6DtpxNgAKoJKA': ['UCKy1dAqELo0zrOtPkf0eTMw',
  'UCpqXJOEqGS-TCnazcHCo0rA',
  'UCClNRixXlagwAd--5MwJKCw',
  'UC2wKfjlioOCLP4xQMOWNcgg',
  'UCq6VFHwMzcMXbuKyG7SQYIg'],
 'UC-Eh1lR_tWVtv9AvxLybMtQ': ['UCByOQJjav0CUDwxCk-jVNRQ',
  'UCeLHszkByNZtPKcaVXOCOQQ',
  'UC0Kgvj5t_c9EMWpEDWJuR1Q',
  'UCcYrdFJF7hmPXRNaWdrko4w',
  'UC6pjHMC4QXMi4llCCjtDXWg'],
 'UC-SV8-bUJfXjrRMnp7F8Wzw': ['UCcgVECVN4OKV6DH1jLkqmcA',
  'UC1KPy3cAAj0i0RIFC_SzjMg',
  'UCilwZiBBfI9X6yiZRzWty8Q',
  'UCEYLdM2bdhmw-TS3c0TjFNw',
  'UCMNmwqCtCSpftrbvR3KkHDA'],
 'UC-lHJZR3Gqxm24_Vd_AJ5Yw': ['UCq6VFHwMzcMXbuKyG7SQYIg',
  'UC7_YxT-KID8kRbqZo7MyscQ',
  'UCo_IB5145EVNcf8hw1Kku7w',
  'UCnmGIkw-KdI0W5siakKPKog',
  'UCS5Oz6CHmeoF7vSad0qqXfw'],
 'UC-yXuc1__OzjwpsJPlxYUCQ': ['UClW4jraMKz6Qj69lJf-tODA',
  'UCByOQJjav0CUDwxCk-jVNRQ',
  'UC-Eh1lR_tWVtv

In [11]:
# correlate chan ids to chan names
chan_names_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chan_names_IDs.csv'

chan_names_ids = pd.read_csv(chan_names_file_path)

chan_usernames_dict = {}
# Traverse the chan_recommendations_dict and correlate IDs to names
for chan_id, recommendations in chan_recommendations_dict.items():
    # Find the corresponding username for the channel ID
    matching_row = chan_names_ids[chan_names_ids['Youtube Url'] == chan_id]
    matching_row = matching_row['username'].values[0]
    chan_usernames_dict[matching_row] = []
    for rec in recommendations:
      #translate the recs too
      translated_rec = chan_names_ids[chan_names_ids['Youtube Url'] == rec]
      chan_usernames_dict[matching_row].append(translated_rec['username'].values[0])

# Now chan_usernames_dict contains the mapping of channel IDs to usernames
print(chan_usernames_dict)


{'Emily Tube': ['Toys and Colors', 'Ya - Alisa', '✿ Kids Diana Show', "Ryan's World", '★ Kids Roma Show'], 'PlayStation': ['IGNentertainment', 'theRadBrad', 'H2ODelirious', 'typicalgamer', 'penguinz0'], 'TygaTygaTV': ['DrakeOfficial', 'postmalone', 'DJ Khaled', 'ChrisBrownTV', 'Lil Pump'], 'RomanAtwoodVlogs': ['JakePaulProductions', 'VitalyzdTv', 'oRugrat', 'Patty Mayo', 'aynakoitsroi'], 'PewDiePie': ['penguinz0', 'markiplierGAME', 'MatthewPatrick13', 'Ryan Trahan', 'TheDiamondMinecart'], 'WorldStarHipHopTV': ['YoungBoy Never Broke Again', 'DrakeOfficial', 'TygaTygaTV', 'GooodLifeFilms', 'DJ Khaled'], 'Crafty Panda': ['Troom Troom', '5-Minute Crafts LIKE', '123 GO!', '5-Minute Recipes', '5-Minute Crafts Recycle'], 'DJ Khaled': ['DrakeOfficial', 'TygaTygaTV', 'Cardi B', 'ChrisBrownTV', 'WorldStarHipHopTV'], 'Doctor Mike': ['watchcut', 'BuzzFeedVideo', 'SSSniperWolf', 'Daily Dose Of Internet', 'Ryan Trahan'], 'xoxxxoooxo': ['postmalone', 'SAM SMITH', 'taylorgangent', 'Camila Cabello', 'H

In [13]:
df_channels_usernames = pd.DataFrame(list(chan_usernames_dict.items()), columns=['Channel', 'Recommendations'])

# Save the DataFrame to a CSV file
csv_file_path = 'recomm_recs.csv'
df_channels_usernames.to_csv(csv_file_path, index=False)